In [9]:
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.core import SimpleDirectoryReader,  VectorStoreIndex, ServiceContext
import os

In [22]:
from llama_index.core.memory import (
    VectorMemory,
    SimpleComposableMemory,
    ChatMemoryBuffer,
)

In [10]:
from llama_index.core import StorageContext, VectorStoreIndex, load_index_from_storage
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import ServiceContext
from llama_index.llms.groq import Groq

In [11]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# Define your embedding model from Hugging Face
embedding_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [12]:
llama2 = Groq(model="llama-3.1-8b-instant", api_key="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",stream=True)
service_context = ServiceContext.from_defaults(embed_model=embedding_model, llm=llama2)

/tmp/ipykernel_80810/4069274112.py:2: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embedding_model, llm=llama2)


In [6]:
data_dir = "python_tutorials22"

In [7]:
documents = SimpleDirectoryReader(data_dir).load_data()

In [8]:
import numpy as np
document_embeddings = [embedding_model.get_text_embedding(doc.text) for doc in documents]

# Convert to numpy array
embeddings_matrix = np.array(document_embeddings).astype("float32")

KeyboardInterrupt: 

In [ ]:

import faiss
dimension = embeddings_matrix.shape[1]  # Dimension of the embeddings
faiss_index = faiss.IndexFlatL2(dimension)
faiss_index.add(embeddings_matrix)

# Initialize FAISSVectorStore with the faiss_index
vector_store = FaissVectorStore(faiss_index=faiss_index)

In [ ]:
index = VectorStoreIndex.from_documents(
    documents, vector_store=vector_store, service_context=service_context
)

In [ ]:
index.storage_context.persist(persist_dir="./python_doc_index")

In [23]:
vector_memory = VectorMemory.from_defaults(
    vector_store=None,  # Using default in-memory vector store
    embed_model=embedding_model,
    retriever_kwargs={"similarity_top_k": 1},
)
chat_memory_buffer = ChatMemoryBuffer.from_defaults()
composable_memory = SimpleComposableMemory.from_defaults(
    primary_memory=chat_memory_buffer,
    secondary_memory_sources=[vector_memory],
)

In [13]:
index_load = load_index_from_storage(
            StorageContext.from_defaults(persist_dir="./python_doc_index"),
            service_context=service_context
        )

In [24]:
query_engine=index_load.as_query_engine(ServiceContext=service_context,streaming=True,similarity_top_k=1,memory=composable_memory)
query_engine

In [29]:
streaming_response = query_engine.query("how to give you memory")

# Print the response as it arrives
streaming_response.print_response_stream()

# Alternatively, if you need to do something with each part of the response:
for text in streaming_response.response_gen:
    # Process each chunk of text as it arrives
    print(text, end='', flush=True)

You can give me memory by allocating it for me. This can be done using a memory allocator, which is a function or a system that manages memory allocation and deallocation. In the context of programming, memory allocators are often used to dynamically allocate memory for objects, data structures, and other resources.

To give me memory, you can use a memory allocator to allocate a block of memory that is large enough to hold the data or resources that I need. You can then pass this allocated memory to me, and I can use it to store and manage my data.

In some cases, you may also need to specify the type of memory that you want to allocate, such as heap memory or stack memory. You may also need to consider factors such as memory alignment, padding, and fragmentation when allocating memory.

It's worth noting that the specific steps for allocating memory will depend on the programming language and environment that you are using, as well as the specific requirements of your application.

In [15]:
# Assuming `index` is already created and query engine is set up for streaming
query_engine = index.as_query_engine(streaming=True, similarity_top_k=1)

# Function to format and print streamed Markdown text
def format_and_print_markdown(text_chunk):
    # Example Markdown formatting:
    # Adding bold to headings (assuming headings are capitalized words)
    formatted_text = text_chunk.replace('**', '**')
    formatted_text = text_chunk.replace('*', '*')
    formatted_text = text_chunk.replace('\n', '\n\n')
    print(formatted_text, end='', flush=True)

# Query the index and get a streaming response
streaming_response = query_engine.query("different data types and make a table")

# Stream the response with Markdown formatting
for text_chunk in streaming_response.response_gen:
    format_and_print_markdown(text_chunk)

# # Optionally, print the full response after streaming
# print("\n\nFull Formatted Response:\n" + "-"*50)
# print(streaming_response.response.strip())  # Strip any leading/trailing whitespace


NameError: name 'index' is not defined

In [21]:
streaming_response = query_engine.query("different data types and make a markdown table")

# Print the response as it arrives
streaming_response.print_response_stream()

# Alternatively, if you need to do something with each part of the response:
for text in streaming_response.response_gen:
    # Process each chunk of text as it arrives
    print(text, end='', flush=True)

| Data Type | Description |
|-----------|-------------|
| int       | Whole numbers, e.g. 1, 2, 3, etc. |
| float     | Decimal numbers, e.g. 3.14, -0.5, etc. |
| str       | Strings of characters, e.g. "hello", 'hello', etc. |
| bool      | Boolean values, e.g. True, False, etc. |
| list      | Ordered collections of items, e.g. [1, 2, 3], ["a", "b", "c"], etc. |
| dict      | Unordered collections of key-value pairs, e.g. {"name": "John", "age": 30}, etc. |
| set       | Unordered collections of unique items, e.g. {1, 2, 3}, {"a", "b", "c"}, etc. |
| tuple     | Ordered, immutable collections of items, e.g. (1, 2, 3), ("a", "b", "c"), etc. |

In [81]:
from IPython.display import Markdown, display

In [119]:
query = "difference between iterator and generator and give example code too"
response = query_engine.query(query)
# display(Markdown(response))
print(response)

An iterator and a generator are both used to traverse through a sequence of data, but they differ in their approach and implementation.

An iterator is an object that allows you to traverse through a sequence of data, such as a list or a tuple, one element at a time. It uses the `__iter__()` method to return the iterator object itself, and the `__next__()` method to return the next element in the sequence.

A generator, on the other hand, is a special type of iterator that can be used to generate a sequence of data on-the-fly, without having to store the entire sequence in memory. It uses the `yield` keyword to produce a value, and then pauses its execution until the next value is requested.

Here's an example code that demonstrates the difference between an iterator and a generator:

```python
# Iterator example
class MyIterator:
    def __init__(self, data):
        self.data = data
        self.index = 0

    def __iter__(self):
        return self

    def __next__(self):
        i

In [7]:
from llama_index.core import SimpleDirectoryReader, GPTVectorStoreIndex, Document

# Convert scraped data to LlamaIndex Document format
def convert_to_llama_documents(scraped_docs):
    documents = []
    for doc in scraped_docs:
        documents.append(Document(text=doc['content'], metadata={"title": doc['title']}))
    return documents

# Convert Python docs to LlamaIndex documents
python_documents = convert_to_llama_documents(python_docs)

# Similarly, scrape and convert docs for other languages
# c_cpp_documents = convert_to_llama_documents(scrape_c_cpp_docs())
# java_documents = convert_to_llama_documents(scrape_java_docs())
# javascript_documents = convert_to_llama_documents(scrape_javascript_docs())

# Index the documents using GPTSimpleVectorIndex
index = GPTVectorStoreIndex(python_documents,embed_model=embedding_model)

# Save the index to disk for later use
index.storage_context.persist(persist_dir="./python_docs")

# Query the index (Example)



In [32]:
print(index)

In [27]:
index_load = load_index_from_storage(
            StorageContext.from_defaults(persist_dir="./python_docs"),
            service_context=service_context
        )

In [30]:
index_load

In [28]:
query_engine=index_load.as_query_engine(ServiceContext=service_context)
query_engine

In [29]:
query = "How do I install Python packages?"
response = query_engine.query(query)
print(response)

Empty Response


[]

In [9]:
from llama_index.core import StorageContext,load_index_from_storage

In [13]:
index_new = load_index_from_storage(
            StorageContext.from_defaults(persist_dir="./python_docs"),
            embed_model=embedding_model
        )

In [14]:
from llama_index.core import StorageContext, VectorStoreIndex, load_index_from_storage
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import ServiceContext
from llama_index.llms.groq import Groq

In [15]:
llama2 = Groq(model="llama-3.1-8b-instant", api_key="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")
service_context = ServiceContext.from_defaults(embed_model=embedding_model, llm=llama2)


/tmp/ipykernel_26965/4285167251.py:2: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embedding_model, llm=llama2)


In [17]:
query_engine=index_new.as_query_engine(llm=llama2)
query_engine

In [18]:
query = "How do I install Python packages?"
response = query_engine.query(query)
print(response)

Empty Response


AttributeError: 'VectorStoreIndex' object has no attribute 'query'